# Using `prompto` with Ollama

In [1]:
from prompto.settings import Settings
from prompto.experiment import Experiment
from dotenv import load_dotenv
import os

When using `prompto` to query models from the Ollama API, lines in our experiment `.jsonl` files must have `"api": "ollama"` in the prompt dict. 

## Setting up Ollama locally

In this notebook, we assume that you have a local instance of the Ollama API running. For installing Ollama, please refer to the [Ollama documentation](https://ollama.com/). Once you have it installed and have it running, e.g. with `ollama serve` in the terminal, you can proceed with the following steps.


By default, the address and port that Ollama uses when running is `localhost:11434`. When developing this notebook, we were running Ollama locally so we set the `OLLAMA_API_ENDPOINT` to `http://localhost:11434`. If you are running the server at a different address or port, you can specify with the `OLLAMA_API_ENDPOINT` environment variable accordingly as described below.

## Downloading models

In this notebook and our example experiment file ([data/input/ollama-example.jsonl](./data/input/ollama-example.jsonl)), we have set to query from Llama 3, phi-3 and Gemma models - note that Ollama defaults to the smaller versions of these (8B, 3B, 2B). You can download these models using the following commands in the terminal:
```bash
ollama pull llama3
ollama pull phi3
ollama pull gemma
```

If you'd prefer to query other models, you can replace the model names in the experiment file with the models you have downloaded. We simply return an error if the model is not found in the Ollama endpoint that is running.

## Environment variables

For the [Ollama API](../../docs/models.md#ollama), there are two environment variables that could be set:
- `OLLAMA_API_ENDPOINT`: the API key for the Ollama API

As mentioned in the [model docs](../../docs/models.md#model-specific-environment-variables), there are also model-specific environment variables too which can be utilised. In particular, if you specify a `model_name` key in a prompt dict, one could also specify a `OLLAMA_API_ENDPOINT_model_name` environment variable to indicate the API key used for that particular model (where "model_name" is replaced to whatever the corresponding value of the `model_name` key is). We will see a concrete example of this later.

To set environment variables, one can simply have these in a `.env` file which specifies these environment variables as key-value pairs:
```
OLLAMA_API_ENDPOINT=<YOUR-OLLAMA-ENDPOINT>
```

If you make this file, you can run the following which should return `True` if it's found one, or `False` otherwise:

In [2]:
load_dotenv(dotenv_path=".env")

True

Now, we obtain those values. We raise an error if the `OLLAMA_API_ENDPOINT` environment variable hasn't been set:

In [3]:
OLLAMA_API_ENDPOINT = os.environ.get("OLLAMA_API_ENDPOINT")
if OLLAMA_API_ENDPOINT is None:
    raise ValueError("OLLAMA_API_ENDPOINT is not set")
else:
    print(f"Using OLLAMA_API_ENDPOINT: {OLLAMA_API_ENDPOINT}")

Using OLLAMA_API_ENDPOINT: http://localhost:11434


If you get any errors or warnings in the above two cells, try to fix your `.env` file like the example we have above to get these variables set.

## Types of prompts

With the Ollama API, the prompt (given via the `"prompt"` key in the prompt dict) can take several forms:
- a string: a single prompt to obtain a response for
- a list of strings: a sequence of prompts to send to the model
    - this is useful in the use case of simulating a conversation with the model by defining the user prompts sequentially
- a list of dictionaries with keys "role" and "content", where "role" is one of "user", "assistant", or "system" and "content" is the message
    - this is useful in the case of passing in some conversation history or to pass in a system prompt to the model

We have created an input file in [data/input/ollama-example.jsonl](./data/input/ollama-example.jsonl) with an example of each of these cases as an illustration.

In [4]:
settings = Settings(data_folder="./data", max_queries=12)
experiment = Experiment(file_name="ollama-example.jsonl", settings=settings)

We set `max_queries` to 12 so we send 12 queries a minute (every 5 seconds).

In [5]:
print(settings)

Settings: data_folder=./data, max_queries=12, max_attempts=3, parallel=False
Subfolders: input_folder=./data/input, output_folder=./data/output, media_folder=./data/media


In [6]:
len(experiment.experiment_prompts)

6

We can see the prompts that we have in the `experiment_prompts` attribute:

In [7]:
experiment.experiment_prompts

[{'id': 3,
  'api': 'ollama',
  'model_name': 'gemma',
  'prompt': ['How does international trade create jobs?',
   'I want a joke about that'],
  'parameters': {'temperature': 1, 'num_predict': 100, 'seed': 0}},
 {'id': 4,
  'api': 'ollama',
  'model_name': 'gemma',
  'prompt': [{'role': 'system',
    'content': 'You are a helpful assistant designed to answer questions briefly.'},
   {'role': 'user',
    'content': 'What efforts are being made to keep the hakka language alive?'}],
  'parameters': {'temperature': 1, 'num_predict': 100, 'seed': 0}},
 {'id': 5,
  'api': 'ollama',
  'model_name': 'gemma',
  'prompt': [{'role': 'system',
    'content': 'You are a helpful assistant designed to answer questions briefly.'},
   {'role': 'user', 'content': "Hello, I'm Bob and I'm 6 years old"},
   {'role': 'assistant', 'content': 'Hi Bob, how may I assist you?'},
   {'role': 'user', 'content': 'How old will I be next year?'}],
  'parameters': {'temperature': 1, 'num_predict': 100, 'seed': 0}},


- In the first prompt (`"id": 0`), we have a `"prompt"` key which is a string and we specify a `"model_name"` key to be "llama3".
- In the second prompt (`"id": 1`), we have a `"prompt"` key which is a string and we specify a `"model_name"` key to be "phi".
- In the third prompt (`"id": 2`), we have a `"prompt"` key which is a string and we specify a `"model_name"` key to be "unknown-model". This will give an error as this won't be a model available in the Ollama API (unless you added a custom model of such name). This is to just illustrate that if you specify a model name that doesn't exist, you will get an error.
- In the fourth prompt (`"id": 3`), we have a `"prompt"` key which is a list of strings and we specify a `"model_name"` key to be "gemma".
- In the fifth prompt (`"id": 4`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "content" key. This acts as passing in a system prompt. Here, we just have a system prompt before a user prompt. We specify a `"model_name"` key to be "gemma".
- In the sixth prompt (`"id": 5`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "content" key. Here, we have a system prompt and a series of user/assistant interactions before finally having a user prompt. This acts as passing in a system prompt and conversation history. We specify a `"model_name"` key to be "gemma".

## Running the experiment

We now can run the experiment using the async method `process` which will process the prompts in the input file asynchronously. Note that a new folder named `timestamp-ollama-example` (where "timestamp" is replaced with the actual date and time of processing) will be created in the output directory and we will move the input file to the output directory. As the responses come in, they will be written to the output file and there are logs that will be printed to the console as well as being written to a log file in the output directory.

If you have `ollama serve` running in the terminal, you'll be able to see queries being sent to the Ollama API and responses being received.

In [8]:
responses, avg_query_processing_time = await experiment.process()

Waiting for responses  (attempt 1/3): 100%|██████████| 6/6 [00:09<00:00,  1.56s/query]


We can see that the responses are written to the output file, and we can also see them as the returned object. From running the experiment, we obtain prompt dicts where there is now a `"response"` key which contains the response(s) from the model.

For the case where the prompt is a list of strings, we see that the response is a list of strings where each string is the response to the corresponding prompt.

In [9]:
responses

[{'id': 4,
  'api': 'ollama',
  'model_name': 'gemma',
  'prompt': [{'role': 'system',
    'content': 'You are a helpful assistant designed to answer questions briefly.'},
   {'role': 'user',
    'content': 'What efforts are being made to keep the hakka language alive?'}],
  'parameters': {'temperature': 1, 'num_predict': 100, 'seed': 0},
  'response': '**Efforts to preserve the Hakka language:**\n\n* **Language immersion programs:** Hakka-speaking schools and communities organize programs to promote the language among younger generations.\n\n\n* **Digital preservation:** Recording and archiving Hakka speech, songs, and stories online.\n\n\n* **Government initiatives:** Some governments have implemented policies to support Hakka language preservation and education.\n\n\n* **Community-driven efforts:** Hakka cultural organizations and diaspora groups actively promote the language through workshops, festivals, and online platforms'},
 {'id': 5,
  'api': 'ollama',
  'model_name': 'gemma',

## Running the experiment via the command line

We can also run the experiment via the command line. The command is as follows (assuming that your working directory is the current directory of this notebook, i.e. `examples/ollama`):
```bash
prompto_run_experiment --file data/input/ollama-example.jsonl --max_queries 30
```